In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

import pandas as pd
from modules import Preprocess
from UserParams import Params
from datetime import datetime
# 最大列の表示数を指定
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 30)

## 前処理＆データ結合

In [2]:
# 集計開始年月日
_year = 2007

In [3]:
# インスタンス生成
preprocess = Preprocess()
# 各テーブルの前処理
manager = preprocess.preprocess_rawdata(since_=datetime(_year, 1, 1))
columns_info = manager.get_columns()

In [4]:
# データ内容の確認
_results = manager.__dict__['_DataManager__results']
_raceinfo = manager.__dict__['_DataManager__raceinfo']
_horseresults = manager.__dict__['_DataManager__horseresults']
_paybacks = manager.__dict__['_DataManager__paybacks']
_peds = manager.__dict__['_DataManager__peds']

In [5]:
_results.head()

,枠番,馬番,斤量,horse_id,jockey_id,sex,age,weight,diff,n_horse,rank
race_id,,,,,,,,,,,
201506010202,8,15,54.0,2012104257,01122,牝,3,502.0,4.0,15,1
201506010202,5,10,54.0,2012103079,01075,牝,3,472.0,-2.0,15,2
201506010202,6,11,54.0,2012105480,01029,牝,3,458.0,6.0,15,3
201506010202,7,14,54.0,2012105148,00663,牝,3,474.0,6.0,15,4
201506010202,3,5,54.0,2012103847,01009,牝,3,476.0,6.0,15,5


In [6]:
_raceinfo.head()

,開催,天気,馬場,日付,race_type,around,race_class,course_len
race_id,,,,,,,,
202006030602,中山,曇,稍重,2020-04-12,ダート,右,未勝利,12.0
201807030712,中京,晴,良,2018-07-21,芝,左,unknown,14.0
202005050509,東京,晴,良,2020-11-21,ダート,左,2勝クラス,16.0
202008010810,京都,曇,重,2020-01-25,ダート,右,2勝クラス,18.0
201905040111,東京,晴,良,2019-10-05,芝,左,オープン,16.0


In [7]:
_paybacks.head(11)

,type,matched,price
race_id,,,
200701010101,複勝,4,130
200701010101,三連複,1-4-11,1290
200701010101,馬単,4→11,1050
200701010101,馬連,4-11,340
200701010101,枠連,4-8,320
200701010101,単勝,4,470
200701010101,複勝,11,100
200701010101,ワイド,1-11,340
200701010101,ワイド,1-4,590


In [8]:
_horseresults.head()

,日付,開催,天気,R,頭数,枠番,馬番,斤量,馬場,着順,着差,上り,賞金,weight,diff,race_type,course_len,first_corner,final_corner,mean_corner
horse_id,,,,,,,,,,,,,,,,,,,,
2012101024,2015-07-25,佐賀,晴,3,8,8,8,56.0,稍重,7,2.2,40.0,0.0,460.0,-6.0,ダート,13.0,6,6,4.75
2012101024,2015-07-12,佐賀,曇,1,5,2,2,56.0,不良,2,2.0,41.0,3.5,466.0,2.0,ダート,13.0,1,2,1.25
2012101024,2015-06-13,佐賀,曇,1,8,1,1,56.0,重,4,3.1,40.8,0.8,464.0,16.0,ダート,13.0,3,3,3.00
2012101024,2015-05-10,東京,晴,3,16,7,13,56.0,良,15,6.0,42.6,0.0,448.0,-2.0,ダート,14.0,9,11,10.00
2012101024,2015-04-25,東京,晴,2,16,2,3,54.0,良,15,2.3,39.2,0.0,450.0,8.0,ダート,13.0,3,3,3.00


In [9]:
_peds.head()

,peds_0,peds_1,peds_16
horse_id,,,
2011102707,2002107011,000a00033a,1992110220
2010104274,1995104427,000a0000cc,000a0062e2
2011100102,2002100816,000a00033a,000a0003a2
2013104061,2002100594,000a00033a,000a0062c2
2013103317,1998102595,1989108341,1989104335


In [10]:
columns_info.horseresults.T.set_axis(['対応するcolumn名の意味'], axis='columns')

,対応するcolumn名の意味
description,馬の過去成績テーブル
DATE,日付
PLACE,開催
WEATHER,天気
R,R
N_HORSES,頭数
FRAME_NUMBER,枠番
HORSE_NUMBER,馬番
WEIGHT,斤量
GROUND_STATE,馬場


In [11]:
# 「馬の過去成績」を対象としたデータ結合
horseresults = columns_info.horseresults
target_columns = [horseresults[key][0] for key in Params.TARGET_COLUMNS]
group_columns =  [horseresults[key][0] for key in Params.GROUP_COLUMNS]
merged_data = manager.merge(target_columns, group_columns)

In [12]:
merged_data.df.head()

,枠番,馬番,斤量,horse_id,jockey_id,sex,age,weight,diff,n_horse,rank,開催,天気,馬場,日付,race_type,around,race_class,course_len,peds_0,peds_1,peds_16,着順_mean,着差_mean,賞金_mean,first_corner_mean,final_corner_mean,mean_corner_mean,course_lenfinal_corner_mean,course_lenfirst_corner_mean,course_lenmean_corner_mean,course_len着差_mean,course_len着順_mean,course_len賞金_mean,race_typefinal_corner_mean,race_typefirst_corner_mean,race_typemean_corner_mean,race_type着差_mean,race_type着順_mean,race_type賞金_mean,開催final_corner_mean,開催first_corner_mean,開催mean_corner_mean,開催着差_mean,開催着順_mean,開催賞金_mean,上り_mean,race_type上り_mean,開催上り_mean,course_len上り_mean
race_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
200701010107,6,7,53.0,2004104659,01102,牡,3,460.0,4.0,11,1,札幌,晴,稍重,2007-08-11,ダート,右,unknown,10.0,000a000013,000a001607,000a009232,6.0,1.1,110.454545,2.818182,2.909091,2.863636,2.75,2.75,2.75,0.45,4.75,193.75,1.625,1.625,1.625,0.8125,4.5,129.375,0.0,0.0,0.0,0.0,0.0,0.0,37.4,37.6375,0.0,36.6
200701010107,5,5,55.0,2003105149,01093,牝,4,470.0,0.0,11,2,札幌,晴,稍重,2007-08-11,ダート,右,unknown,10.0,000a000d6b,000a001beb,000a0092cf,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.000,0.000,0.0000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0
200701010107,3,3,52.0,2003102358,01109,牝,4,482.0,0.0,11,3,札幌,晴,稍重,2007-08-11,ダート,右,unknown,10.0,000a000134,000a001607,000a0094a9,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.000,0.000,0.0000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0
200803030504,3,5,52.0,2003102358,01118,牝,5,498.0,2.0,15,8,福島,晴,良,2008-11-08,ダート,右,unknown,11.5,000a000134,000a001607,000a0094a9,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.000,0.000,0.0000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0
200907010603,1,1,52.0,2003102358,01118,牝,6,496.0,2.0,15,3,中京,晴,稍重,2009-01-25,ダート,左,unknown,10.0,000a000134,000a001607,000a0094a9,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.000,0.000,0.0000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0


In [13]:
merged_data.df.columns

Index(['枠番', '馬番', '斤量', 'horse_id', 'jockey_id', 'sex', 'age', 'weight',
       'diff', 'n_horse', 'rank', '開催', '天気', '馬場', '日付', 'race_type',
       'around', 'race_class', 'course_len', 'peds_0', 'peds_1', 'peds_16',
       '着順_mean', '着差_mean', '賞金_mean', 'first_corner_mean',
       'final_corner_mean', 'mean_corner_mean', 'course_lenfinal_corner_mean',
       'course_lenfirst_corner_mean', 'course_lenmean_corner_mean',
       'course_len着差_mean', 'course_len着順_mean', 'course_len賞金_mean',
       'race_typefinal_corner_mean', 'race_typefirst_corner_mean',
       'race_typemean_corner_mean', 'race_type着差_mean', 'race_type着順_mean',
       'race_type賞金_mean', '開催final_corner_mean', '開催first_corner_mean',
       '開催mean_corner_mean', '開催着差_mean', '開催着順_mean', '開催賞金_mean', '上り_mean',
       'race_type上り_mean', '開催上り_mean', 'course_len上り_mean'],
      dtype='object')

In [14]:
merged_data.df.describe()

,枠番,馬番,斤量,age,weight,diff,n_horse,rank,course_len,着順_mean,着差_mean,賞金_mean,first_corner_mean,final_corner_mean,mean_corner_mean,course_lenfinal_corner_mean,course_lenfirst_corner_mean,course_lenmean_corner_mean,course_len着差_mean,course_len着順_mean,course_len賞金_mean,race_typefinal_corner_mean,race_typefirst_corner_mean,race_typemean_corner_mean,race_type着差_mean,race_type着順_mean,race_type賞金_mean,開催final_corner_mean,開催first_corner_mean,開催mean_corner_mean,開催着差_mean,開催着順_mean,開催賞金_mean,上り_mean,race_type上り_mean,開催上り_mean,course_len上り_mean
count,45369.000000,45369.000000,45369.000000,45369.00000,45369.000000,45369.000000,45369.000000,45369.000000,45369.000000,45369.000000,45369.000000,45369.000000,45369.000000,45369.000000,45369.000000,45369.000000,45369.000000,45369.000000,45369.000000,45369.000000,45369.000000,45369.000000,45369.000000,45369.000000,45369.000000,45369.000000,45369.000000,45369.000000,45369.000000,45369.000000,45369.000000,45369.000000,45369.000000,45369.000000,45369.000000,45369.000000,45369.000000
mean,4.751901,7.998700,55.634541,3.79087,472.231876,0.371267,13.890035,7.443695,18.074880,0.759139,0.145798,11.965965,0.724069,0.712154,0.718897,0.416564,0.433198,0.426797,0.085222,0.416853,10.667583,0.606058,0.626994,0.617823,0.146642,0.617705,13.776097,0.493338,0.505642,0.500213,0.099194,0.513181,10.038269,3.462506,2.795195,2.246263,2.024625
std,2.259141,4.473957,2.214565,1.44916,29.737724,6.661062,2.160089,4.189277,6.200224,2.384046,0.513682,83.496911,2.317930,2.273046,2.291429,1.814318,1.879284,1.842113,0.488462,1.793410,128.901016,2.120233,2.195797,2.153076,0.684240,2.133340,102.100581,2.025995,2.076139,2.042773,0.484426,2.075044,104.278793,10.743405,9.632705,8.739994,8.344003
min,1.000000,1.000000,48.000000,2.00000,334.000000,-36.000000,4.000000,1.000000,10.000000,0.000000,-2.300000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.300000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.700000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,4.000000,54.000000,3.00000,452.000000,-4.000000,13.000000,4.000000,14.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.000000,8.000000,56.000000,3.00000,472.000000,0.000000,15.000000,7.000000,17.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,7.000000,12.000000,57.000000,5.00000,492.000000,4.000000,15.000000,11.000000,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,8.000000,18.000000,65.000000,13.00000,640.000000,48.000000,18.000000,18.000000,42.600000,18.000000,8.900000,4562.662500,18.000000,18.000000,17.500000,18.000000,18.000000,18.000000,21.850000,18.000000,11405.500000,18.000000,18.000000,18.000000,21.500000,18.000000,5436.085714,18.000000,18.000000,18.000000,15.466667,18.000000,8852.100000,46.100000,46.100000,46.100000,52.500000


In [15]:
table_df = merged_data.table
table_df

,description,FRAME_NUMBER,HORSE_NUMBER,WEIGHT,horse_id,jockey_id,sex,age,weight,diff,n_horse,rank,PLACE,WEATHER,GROUND_STATE,DATE,race_type,around,race_class,course_len,peds_0,peds_1,peds_16,着順_mean,着差_mean,賞金_mean,first_corner_mean,final_corner_mean,mean_corner_mean,course_lenfinal_corner_mean,course_lenfirst_corner_mean,course_lenmean_corner_mean,course_len着差_mean,course_len着順_mean,course_len賞金_mean,race_typefinal_corner_mean,race_typefirst_corner_mean,race_typemean_corner_mean,race_type着差_mean,race_type着順_mean,race_type賞金_mean,開催final_corner_mean,開催first_corner_mean,開催mean_corner_mean,開催着差_mean,開催着順_mean,開催賞金_mean,上り_mean,race_type上り_mean,開催上り_mean,course_len上り_mean
0,結合後テーブル,枠番,馬番,斤量,horse_id,jockey_id,sex,age,weight,diff,n_horse,rank,開催,天気,馬場,日付,race_type,around,race_class,course_len,peds_0,peds_1,peds_16,着順_mean,着差_mean,賞金_mean,first_corner_mean,final_corner_mean,mean_corner_mean,course_lenfinal_corner_mean,course_lenfirst_corner_mean,course_lenmean_corner_mean,course_len着差_mean,course_len着順_mean,course_len賞金_mean,race_typefinal_corner_mean,race_typefirst_corner_mean,race_typemean_corner_mean,race_type着差_mean,race_type着順_mean,race_type賞金_mean,開催final_corner_mean,開催first_corner_mean,開催mean_corner_mean,開催着差_mean,開催着順_mean,開催賞金_mean,上り_mean,race_type上り_mean,開催上り_mean,course_len上り_mean


In [16]:
# ラベルエンコーディング＆ダミー変数化の処理による特徴量生成
functions = {
    'dumminize':        {'columns': []},
    'label_encoding':   {'columns': [table_df[key][0] for key in Params.CATEGORY_COLUMNS]},
    'encode_jockey_id': {},
    'encode_horse_id':  {},
    'drop':             {'columns': table_df['DATE'].to_list()},
}
inputs = manager.get_features(merged_data.df, functions)

In [17]:
inputs.features

,枠番,馬番,斤量,horse_id,jockey_id,sex,age,weight,diff,n_horse,rank,開催,天気,馬場,race_type,around,race_class,course_len,peds_0,peds_1,peds_16,着順_mean,着差_mean,賞金_mean,first_corner_mean,final_corner_mean,mean_corner_mean,course_lenfinal_corner_mean,course_lenfirst_corner_mean,course_lenmean_corner_mean,course_len着差_mean,course_len着順_mean,course_len賞金_mean,race_typefinal_corner_mean,race_typefirst_corner_mean,race_typemean_corner_mean,race_type着差_mean,race_type着順_mean,race_type賞金_mean,開催final_corner_mean,開催first_corner_mean,開催mean_corner_mean,開催着差_mean,開催着順_mean,開催賞金_mean,上り_mean,race_type上り_mean,開催上り_mean,course_len上り_mean
race_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
200701010107,6,7,53.0,4553,156,2,3,460.0,4.0,11,1,6,2,1,0,0,3,10.0,1,184,458,6.0,1.1,110.454545,2.818182,2.909091,2.863636,2.75,2.75,2.75,0.45,4.75,193.75,1.625,1.625,1.625,0.8125,4.5,129.375,0.0,0.0,0.0,0.0,0.0,0.0,37.4,37.6375,0.0,36.6
200701010107,5,5,55.0,3229,148,1,4,470.0,0.0,11,2,6,2,1,0,0,3,10.0,141,241,465,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.000,0.000,0.0000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0
200701010107,3,3,52.0,2847,163,1,4,482.0,0.0,11,3,6,2,1,0,0,3,10.0,45,184,480,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.000,0.000,0.0000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0
200803030504,3,5,52.0,2847,172,1,5,498.0,2.0,15,8,8,2,2,0,0,3,11.5,45,184,480,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.000,0.000,0.0000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0
200907010603,1,1,52.0,2847,172,1,6,496.0,2.0,15,3,0,2,1,0,1,3,10.0,45,184,480,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.000,0.000,0.0000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202310010609,8,14,56.0,25168,148,2,3,510.0,18.0,13,9,4,3,3,0,0,0,17.0,19135,1749,2210,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.000,0.000,0.0000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0
202310010609,7,11,54.0,25131,231,1,3,462.0,-8.0,13,10,4,3,3,0,0,0,17.0,972,748,966,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.000,0.000,0.0000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0
202310010609,4,5,56.0,25197,146,2,3,478.0,6.0,13,11,4,3,3,0,0,0,17.0,6199,1724,1675,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.000,0.000,0.0000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0


In [18]:
inputs.features.columns

Index(['枠番', '馬番', '斤量', 'horse_id', 'jockey_id', 'sex', 'age', 'weight',
       'diff', 'n_horse', 'rank', '開催', '天気', '馬場', 'race_type', 'around',
       'race_class', 'course_len', 'peds_0', 'peds_1', 'peds_16', '着順_mean',
       '着差_mean', '賞金_mean', 'first_corner_mean', 'final_corner_mean',
       'mean_corner_mean', 'course_lenfinal_corner_mean',
       'course_lenfirst_corner_mean', 'course_lenmean_corner_mean',
       'course_len着差_mean', 'course_len着順_mean', 'course_len賞金_mean',
       'race_typefinal_corner_mean', 'race_typefirst_corner_mean',
       'race_typemean_corner_mean', 'race_type着差_mean', 'race_type着順_mean',
       'race_type賞金_mean', '開催final_corner_mean', '開催first_corner_mean',
       '開催mean_corner_mean', '開催着差_mean', '開催着順_mean', '開催賞金_mean', '上り_mean',
       'race_type上り_mean', '開催上り_mean', 'course_len上り_mean'],
      dtype='object')

## 学習
T.B.D